# 3. fáza

----------------------------------------------------
Základné importy knižníc:

In [36]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score

pd.options.mode.chained_assignment = None

sns.set_theme(style="whitegrid", palette="viridis")
sns.set_palette("viridis")

Načítanie datasetov zo súborov:

In [37]:
X_test_data = pd.read_csv("dataset/transformed_X_test.csv", sep=",")
X_test_data = X_test_data.iloc[:, 1:]
X_test_data.head(15)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,0.0,0.483019,0.367739,0.471250,0.228984,0.521895,0.367081,0.469825,0.513934,0.452861,0.713273,0.311295,0.179349,0.585875,0.227821,0.751744,0.0
1,1.0,0.236195,0.408903,0.454163,0.851137,0.955975,0.891132,0.418414,0.108259,0.452861,0.378157,0.543411,0.515301,0.634517,0.000000,0.482894,1.0
2,1.0,0.483957,0.614102,0.771766,0.757092,0.690728,0.296679,0.599013,0.250593,0.452861,0.325598,0.500332,0.135756,0.759758,0.227821,0.877955,1.0
3,1.0,0.599179,0.747323,0.868591,0.543235,0.729923,0.385701,0.481822,0.605707,0.000000,0.439048,0.537557,0.568310,0.132010,1.000000,0.751744,1.0
4,0.0,0.423880,0.264049,0.456017,0.458634,0.761094,0.873124,0.263948,0.063787,0.827757,0.382268,0.436688,0.250580,0.635221,1.000000,0.620488,1.0
5,0.0,0.484291,0.615242,0.812390,0.597319,0.413252,0.522909,0.858935,0.876462,0.452861,0.681511,0.583228,0.295508,0.622408,0.473659,0.336909,1.0
6,0.0,0.395645,0.298660,0.363822,0.763665,0.703895,0.822929,0.635185,0.068145,0.241684,0.675150,0.497190,0.604259,0.556999,0.227821,0.482894,0.0
7,1.0,0.713330,0.530216,0.590720,0.105299,0.740337,0.683422,0.761252,0.448814,0.452861,0.443386,0.413512,0.184392,0.352738,1.000000,0.336909,0.0
8,1.0,0.949873,0.346234,0.493798,0.688843,0.646604,0.721638,0.468534,0.335896,0.000000,0.451984,0.467307,0.505322,0.584239,1.000000,0.336909,0.0
9,1.0,0.386364,0.301711,0.479263,0.405125,0.876304,0.632032,0.557463,0.699266,0.000000,0.496654,0.147686,0.564972,0.352850,0.000000,0.178813,1.0


In [38]:
Y_test_data = pd.read_csv("dataset/y_test.csv", sep=",")
Y_test_data = Y_test_data.iloc[:, 1:]
Y_test_data.head(15)

,indicator
0,1
1,1
2,1
3,0
4,1
5,1
6,1
7,0
8,1
9,0


Tu bude implementácia OneR algoritmu.

In [39]:
X_test_data["12"].head(25)

0     0.179349
1     0.515301
2     0.135756
3     0.568310
4     0.250580
5     0.295508
6     0.604259
7     0.184392
8     0.505322
9     0.564972
10    0.310159
11    0.532557
12    0.773292
13    0.830571
14    0.777202
15    0.677716
16    0.777867
17    0.741518
18    0.797980
19    0.488567
20    0.527275
21    0.456222
22    0.544468
23    0.718003
24    0.523530
Name: 12, dtype: float64

In [40]:
def custom_oneR(X_data, Y_data):
    results = []
    X_set = pd.DataFrame(X_data)
    Y_set = np.squeeze(Y_data)

    for i in range(len(X_set.columns)):
        # column = X_set.loc[:, str(i)]
        # print(column)
        # column = X_set[X_set.columns[i]]

        column = set_intervals(X_set[X_set.columns[i]])
        # print(Y_set)
        unique_counts = 0
        # merged_set = pd.concat([X_set[i], Y_set], axis=1)
        merged_set = pd.DataFrame({"variable": column, "label": Y_set})
        most_freq_val = pd.crosstab(column, Y_set).idxmax(axis=1)
        # uniquecounts = [0 for  in range(len(X_set[i].unique()))]

        y_true, y_pred = [], []

        for index, row in merged_set.iterrows():
            if row["label"] == most_freq_val[row["variable"]]:
                # unique_counts[unique_counts.index(index)] += 1
                unique_counts += 1
            y_true.append(row["label"])
            y_pred.append(most_freq_val[row["variable"]])

        current_result = {"variable": X_set.columns[i], "accuracy": unique_counts / len(Y_set), "precision":
            precision_score(y_true, y_pred, average='binary'), "recall": recall_score(y_true, y_pred, average='binary')}
        results.append(current_result)

    return results

In [41]:
def set_intervals(data):
    unique_values = data.value_counts().index.tolist()
    if len(unique_values) > len(data) / 2:
        for value in range(len(data)):
            if 0 <= data[value] < 0.2:
                data[value] = 0.1
            elif 0.2 <= data[value] < 0.4:
                data[value] = 0.3
            elif 0.4 <= data[value] < 0.6:
                data[value] = 0.5
            elif 0.6 <= data[value] < 0.8:
                data[value] = 0.7
            else:
                data[value] = 0.9

    return data

In [42]:
d = {'outlook': [0, 0, 0.5, 1, 1, 1, 0.5, 0, 0, 1, 0, 0.5, 0.5, 1]}
X_frame = pd.DataFrame(d)
Y_frame = pd.Series([0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0])
print(Y_frame)
print(X_frame)

0     0
1     0
2     1
3     1
4     1
5     0
6     1
7     0
8     1
9     1
10    1
11    1
12    1
13    0
dtype: int64
    outlook
0       0.0
1       0.0
2       0.5
3       1.0
4       1.0
5       1.0
6       0.5
7       0.0
8       0.0
9       1.0
10      0.0
11      0.5
12      0.5
13      1.0


In [43]:
res = custom_oneR(X_test_data, Y_test_data)
for index in range(len(res)):
    print(res[index])

{'variable': '0', 'accuracy': 0.6466890192791283, 'precision': 0.6466890192791283, 'recall': 1.0}
{'variable': '1', 'accuracy': 0.6466890192791283, 'precision': 0.6466890192791283, 'recall': 1.0}
{'variable': '2', 'accuracy': 0.702430846605197, 'precision': 0.6956317519962424, 'recall': 0.9598185353208036}
{'variable': '3', 'accuracy': 0.6663872590108969, 'precision': 0.6819288845591817, 'recall': 0.9073233959818535}
{'variable': '4', 'accuracy': 0.6466890192791283, 'precision': 0.6466890192791283, 'recall': 1.0}
{'variable': '5', 'accuracy': 0.7338642078792958, 'precision': 0.7178502879078695, 'recall': 0.9695398574206092}
{'variable': '6', 'accuracy': 0.6466890192791283, 'precision': 0.6466890192791283, 'recall': 1.0}
{'variable': '7', 'accuracy': 0.6466890192791283, 'precision': 0.6466890192791283, 'recall': 1.0}
{'variable': '8', 'accuracy': 0.6466890192791283, 'precision': 0.6466890192791283, 'recall': 1.0}
{'variable': '9', 'accuracy': 0.6466890192791283, 'precision': 0.646689019